In [ ]:
import os
import sys

In [ ]:
home = os.environ['HOME']
work_path = os.path.join(home, 'far/farMe/landau')
src_path = os.path.join(home, 'far/PHARE')
build_path = os.path.join(home, 'far/builds/release/landau')

In [ ]:
sys.path.append(os.path.join(src_path, "pyphare"))

In [ ]:
import subprocess
import pyphare
import matplotlib.pyplot as plt
from pyphare.pharesee.run import Run
from pyphare.core.operators import dot, cross, sqrt, modulus, grad
from pyphare.core.ufuncs import gF, peakIds
import numpy as np
from numpy import polyfit

In [ ]:
Ti = 0.1
Te = 0.2
run_name = "ld_a"

In [ ]:
run_path = os.path.join(work_path, run_name)

In [ ]:
if os.path.isdir(run_path):
    files = os.listdir(run_path)
else:
    files = []

In [ ]:
if 'PYTHONPATH' in os.environ:
    os.environ['PYTHONPATH'] += os.pathsep + os.path.join(src_path, "pyphare")
else:
    os.environ['PYTHONPATH'] = os.pathsep + os.path.join(src_path, "pyphare")
os.environ['PYTHONPATH'] += os.pathsep + build_path

In [ ]:
if 'EM_E.h5' not in files :
    os.chdir(work_path)
    # subprocess.run(['mpirun', '-n', '2', '/usr/bin/python3', work_path+'/ld.py', run_name, str(Te), str(Ti)], env=os.environ)
    subprocess.run(['/usr/bin/python3', work_path+'/ld_ai.py', run_name, str(Te), str(Ti)], env=os.environ)

In [ ]:
run  = Run(run_path)

In [ ]:
times = np.arange(0, 33, 2)

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
fig, ax = plt.subplots(figsize=(12,2))

for i, time in enumerate(times):
    N = run.GetNi(time)
    n = gF(N, sigma=6)
    # V = run.GetVi(time)
    # v = gF(V, sigma=6)

    n.plot(qty='value', ax=ax, ls='solid', lw=2.0, color=colors[i%10], ylabel='Density')

In [ ]:
zob = run.GetE(0, merged=True)
print(type(zob), zob)

In [ ]:
zob = run.GetMassDensity(0, merged=True)

In [ ]:
zob = run.GetNi(0, merged=True)

In [ ]:
# zob = run.GetN(0, "protons")

In [ ]:
def sine_func(x, A, B, C, D):
    return A * np.sin(B * x + C) + D

from scipy.optimize import curve_fit

Amp = np.zeros(times.shape)
for i, tim in enumerate(times):
    initial_guess = [0.1, 0.157, 0.0, 1.0]
    dn_func, x_fin = run.GetNi(tim, merged=True)['rho']
    x_ = x_fin[0]
    y_ = dn_func(x_)
    params, covariance = curve_fit(sine_func, x_, y_, p0=initial_guess)
    # print(params)
    A_fit, B_fit, C_fit, D_fit = params
    Amp[i] = np.fabs(A_fit)
    print(A_fit)

In [ ]:
Ti = 0.1
Te = 0.2
run_name = "ld_b"

In [ ]:
run_path = os.path.join(work_path, run_name)

In [ ]:
if os.path.isdir(run_path):
    files = os.listdir(run_path)
else:
    files = []

In [ ]:
if 'PYTHONPATH' in os.environ:
    os.environ['PYTHONPATH'] += os.pathsep + os.path.join(src_path, "pyphare")
else:
    os.environ['PYTHONPATH'] = os.pathsep + os.path.join(src_path, "pyphare")
os.environ['PYTHONPATH'] += os.pathsep + build_path

In [ ]:
if 'EM_B.h5' not in files :
    os.chdir(work_path)
    # subprocess.run(['mpirun', '-n', '2', '/usr/bin/python3', work_path+'/ld.py', run_name, str(Te), str(Ti)], env=os.environ)
    subprocess.run(['/usr/bin/python3', work_path+'/ld_kaw.py', run_name, str(Te), str(Ti)], env=os.environ)

In [ ]:
run  = Run(run_path)

In [ ]:
times = np.arange(0, 81, 4)

In [ ]:
fig, ax = plt.subplots(figsize=(12,2))

for i, time in enumerate(times[6:10]):
    V = run.GetVi(time)
    B = run.GetB(time)
    
    B.plot(qty='y', ax=ax, ls='solid', lw=2.0, color=colors[i%10], ylabel='$B_y$')

In [ ]:
by_func, x_fin = run.GetB(time, merged=True)['By']
# type(x_fin[0]), type(by_func(x_fin[0]))

In [ ]:
from scipy.optimize import curve_fit

def sine_func(x, A, B, C, D):
    return A * np.sin(B * x + C) + D

In [ ]:
tim = 10
initial_guess = [1, 1.57, 0.0, 0.0]
by_func, x_fin = run.GetB(tim, merged=True)['By']

In [ ]:
x_ = x_fin[0]
y_ = by_func(x_)
params, covariance = curve_fit(sine_func, x_, y_, p0=initial_guess)
A_fit, B_fit, C_fit, D_fit = params

In [ ]:
params

In [ ]:
Amp = np.zeros(times.shape)
for i, tim in enumerate(times):
    initial_guess = [1, 1.57, 0.0, 0.0]
    by_func, x_fin = run.GetB(tim, merged=True)['By']
    x_ = x_fin[0]
    y_ = by_func(x_)
    params, covariance = curve_fit(sine_func, x_, y_, p0=initial_guess)
    # print(params)
    A_fit, B_fit, C_fit, D_fit = params
    Amp[i] = np.fabs(A_fit)
    print(A_fit)

In [ ]:
Amp

In [ ]:
fig, ax = plt.subplots(figsize=(6,3))

ax.plot(times, Amp)